In [2]:
%%writefile rename_files.py
import os
import urllib.parse
from pathlib import Path
import argparse

def rename_files(folder_path: Path,
                 file_pattern: str,
                 slice_start: int = 0) -> None:
    folder = Path(folder_path)

    if not folder.exists() or not folder.is_dir():
        print(f"Error: {folder} is not a valid folder.")
        return

    cnt = 0

    # Iterate deterministically for nicer logs
    for p in sorted(folder.glob(file_pattern)):
        # Skip directories
        if p.is_dir():
            continue

        original_name = p.name  # str
        # Decode URL-encoded characters
        decoded_name = urllib.parse.unquote(original_name)

        # Optionally remove first N characters (guard against over-slicing)
        if slice_start > 0 and len(decoded_name) > slice_start:
            decoded_name = decoded_name[slice_start:]

        # If name didn't change, skip
        if decoded_name == original_name:
            continue

        if decoded_name == "":
            print(f"⚠ Skipping (empty target name) for: {original_name}")
            continue

        new_path = p.with_name(decoded_name)

        # Prevent collisions
        if new_path.exists():
            print(f"⚠ Skipping (target exists): {decoded_name}")
            continue

        # Perform rename
        p.rename(new_path)
        print(f"Renamed: {original_name} → {decoded_name}")
        cnt += 1

    print(f"✅ Total files renamed: {cnt}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Rename URL-encoded files in a folder")
    parser.add_argument("--folder_path", type=str, required=True, help="Path to the folder")
    parser.add_argument("--file_pattern", type=str, default="*", help="Glob to select files (e.g., '*.txt')")
    parser.add_argument("--slice_start", type=int, default=0,
                        help="Optional: remove first N characters from filename (after decoding)")

    args = parser.parse_args()
    rename_files(args.folder_path, args.file_pattern, args.slice_start)


Overwriting rename_files.py


In [5]:
!python3 rename_files.py --folder_path "/Users/AnirbanGuha/pCloud Drive/My Videos/Adult" --file_pattern "EPORNER*"

Renamed: EPORNER.COM%20-%20[n1uXBbjZIgj]%20Oldschool%20desi%20college%20couples%20making%20out%20in%20rented%20house%20bunking%20classes%20(480).mp4 → EPORNER.COM - [n1uXBbjZIgj] Oldschool desi college couples making out in rented house bunking classes (480).mp4
✅ Total files renamed: 1
